In [0]:
import torch
import torch.nn as nn
import numpy as np

In [0]:
"""
Para implementarmos nossa rede de maneira genérica, esta foi implementada em 
duas partes: de convolução e sequencial. PyTorch não tem uma camada de 'flatter'
que serve para transformar um tensor 3D em um array 1D de números, necessário
para alimentar a saída da camada de convolução para a camada totalmente conecta-
-da (fully conected layer). Este problema e resolvido no método forward(), onde
podemos redimensionar nosso 'batch' de tensores 3D em um 'batch' de vetores 1D
"""
class DQN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DQN, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        conv_out_size = self._get_conv_out(input_shape)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )
    """
    Outro problema é que não sabemos o número exato de valores na saída da
    camada de convolução produzidos a partir do input com dado formato (shape). 
    Entretanto precisamos passar esse número para o construtor da primeira cama-
    -da totalmente conectada. A função _get_conv_out() aceita o formato de entrada
    e aplica a camada de convoluçao em um tensor falso de tal formato. O resultado
    da função será igual ao número de parametros retornados por essa aplicação.
    Sera rápido, visto que essa chamada só ocorre uma vez na criação do modelo,
    mas permitirá que nosso código seja genérico.
    """
    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))
    """
    A função forward() aceita o tensor de entrada 4D (a primeira dimensão é o 
    tamanho da batch, a segunda é o canal de cor e a terceira e quarta são as 
    dimensões da imagem). Primeiro aplicamos a camada de convolução para a entrada
    e então obtemos um tensor 4D de saída. Este resultado é então 'flattened' para
    ter duas dimensões: o tamanho da batch e todos os parâmetros retornados pela
    convolução para essa batch como um longo vetor de números.
    """
    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1)
        return self.fc(conv_out)